In [ ]:
"""
# Jesús Bautista Villar <jesbauti20@gmail.com>
- Distributed estimation of the centroid and the ascending direction -
"""

In [ ]:
# If executed from Google Colab --------------------------------------------------------
# !git clone https://github.com/jesusBV20/source_seeking_distr.git
# !rsync -a source_seeking_distr/ .
# !rm -r source_seeking_distr

# If you want to use latex with matplotlib ---------------------------------------------
# !apt install -y texlive texlive-latex-extra texlive-fonts-recommended dvipng cm-super
# !pip install -y latex

In [ ]:
import numpy as np
import os
import sys
from tqdm import tqdm

# Graphic tools
import matplotlib.pyplot as plt
from matplotlib.legend import Legend # legend artist
from matplotlib import ticker

from IPython.display import HTML
from matplotlib.animation import PillowWriter

# -------------------------------------------------------------------------------------
# Import from the Swarm Systems Lab Simulator

# Tell matplotlib to use latex
from ssl_simulator.visualization import set_paper_parameters
set_paper_parameters(fontsize=12)

# Main utility functions used along the notebook
from ssl_simulator import create_dir, add_src_to_path
from ssl_simulator.math import XY_distrib, gen_Z_random, gen_Z_split
from ssl_simulator.visualization import unicycle_patch
from ssl_simulator.components.scalar_fields import PlotterScalarField

# Scalar field functions
from ssl_simulator.components.scalar_fields import SigmaNonconvex

# -------------------------------------------------------------------------------------
# Import custom functions and classes (src in path is requiered)
add_src_to_path("")

from sim_core.visualization import plot_centroid_estimation, plot_estimation_evolution
from sim_core.visualization import plot_mu_estimation
from sim_core.visualization import anim_centroid_estimation, anim_mu_estimation

# -------------------------------------------------------------------------------------
# Define constants for file paths
OUTPUT_FOLDER = os.path.join("..","output")
create_dir(OUTPUT_FOLDER)

## Centroid estimation

### Rec

In [ ]:
## Parameters #####
r = 1
thetas = np.linspace(0,2*np.pi,4)[:-1]

p = np.array([r*np.cos(thetas), r*np.sin(thetas)]).T
Z = ((0,1), (1,2))

dt = 0.1
its = 10  # iterations per dt

# ----------------------------------------------------------------------
# Plotting
# ----------------------------------------------------------------------

fig = plt.figure(figsize=(10, 6), dpi=100)
ax  = fig.subplots()

for i in range(p.shape[0]):
    print(i,": ", p[i,:])
plot_centroid_estimation(ax, p, Z, dt=dt, tf=0.5, its=its, k=1, 
                         legend=False, xlab=True, ylab=True)

plot_estimation_evolution(p, Z, dt=dt, tf=0.5, its=its, k=1)

# Show the plot!
plt.show()

In [ ]:
## Parameters #####
p = np.array([[1,1],[-1,1],[-1,-1],[1,-1]]) + np.array([1.5,1.5])
Z = ((0,1), (1,2), (2,3), (3,0))

dt = 0.01
its = 10 # iterations per dt

# ----------------------------------------------------------------------
# Plotting
# ----------------------------------------------------------------------

fig = plt.figure(figsize=(18, 10), dpi=100)
ax  = fig.subplots(1,3)

plot_centroid_estimation(ax[0], p, Z, dt=dt, tf=0.1, its=its, k=0.01, 
                         legend=True,xlab=True,ylab=True)
plot_centroid_estimation(ax[1], p, Z, dt=dt, tf=0.1, its=its, k=0.1)
plot_centroid_estimation(ax[2], p, Z, dt=dt, tf=0.1, its=its, k=1)

# Show the plot!
plt.show()

In [ ]:
anim = anim_centroid_estimation(p, Z, dt=0.01, tf=1, its=10, k=0.03)
HTML(anim.to_html5_video()) # It takes a loooot of time...

# writer = PillowWriter(fps=15, bitrate=1800)
# anim.save(os.path.join(OUTPUT_FOLDER, "centroid1.gif"),
#         writer = writer)

### Random

In [ ]:
## Parameters #####
pc = np.array([0,0])
lims = np.array([2,1.8])

N = 10
p = XY_distrib(N,pc,lims)
Z = gen_Z_random(N)

dt = 0.01
its = 1 # iterations per dt

# --------------------------------------------------------------------------------------
# Plotting
# --------------------------------------------------------------------------------------

fig = plt.figure(figsize=(12, 8), dpi=100)
axes  = fig.subplots(1,3)

plot_centroid_estimation(axes[0], p, Z, dt=dt, tf=0.1, its=its, k=0.2, legend_fs=9, sz=40, lw=1.5,
                         legend=True,xlab=True,ylab=True)
plot_centroid_estimation(axes[1], p, Z, dt=dt, tf=0.5, its=its, k=0.2, legend_fs=9, sz=40, lw=1.5)
plot_centroid_estimation(axes[2], p, Z, dt=dt, tf=1, its=its, k=0.2, legend_fs=9, sz=40, lw=1.5)

for ax in axes:
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# Show the plot!
plt.show()

In [ ]:
fps = 30
anim = anim_centroid_estimation(p, Z, dt=0.01, fps=fps, tf=2, its=1, k=0.1)
HTML(anim.to_html5_video()) # It takes a loooot of time...

# writer = PillowWriter(fps=15, bitrate=1800)
# anim.save(os.path.join(OUTPUT_FOLDER, "centroid2.gif"),
#         writer = writer)

## Ascending direction estimation

### Ex 1

In [ ]:
## Parameters #####
p = np.array([[2,1],[-2,1],[-2,-1],[2,-1]]) + np.array([10,1])
Z = [(0,1), (1,2), (2,3), (3,0)]
N = p.shape[0]

dt = 0.1
its = 10 # iterations per dt

# ----------------------------------------------------------------------
# Generating the scalar field
# ----------------------------------------------------------------------
n = 2
max_int = 20
mu = [40,40]
dev = 10

scalar_field = SigmaNonconvex(k=0.04, dev=dev, mu=mu)

# Draw the scalar field
fig = plt.figure(figsize=(5, 5), dpi=60)
ax = fig.subplots()

scalar_field_plotter = PlotterScalarField(scalar_field)
scalar_field_plotter.draw(fig=fig, ax=ax, xlim=70, ylim=70, contour_levels=8)

# Agents
phi = np.pi/3
for n in range(N):
    icon = unicycle_patch(p[n,:], phi, color="royalblue", size=3)
    ax.add_patch(icon)

# ----------------------------------------------------------------------
# Plotting
# ----------------------------------------------------------------------

fig = plt.figure(figsize=(12, 8), dpi=100)
ax  = fig.subplots(1,3)

plot_mu_estimation(ax[0], p, Z, scalar_field, dt=dt, tf=0.1, its=its, k=0.5, 
                         legend=False,xlab=True,ylab=True)
plot_mu_estimation(ax[1], p, Z, scalar_field, dt=dt, tf=0.5, its=its, k=0.5)
plot_mu_estimation(ax[2], p, Z, scalar_field, dt=dt, tf=1, its=its, k=0.5)

arrs = [plt.scatter([],[],c="k",marker=r"$\uparrow$",s=60),
        plt.scatter([],[],c="blue",marker=r"$\uparrow$",s=60),
        plt.scatter([],[],c="red",marker=r"$\uparrow$",s=60)]

labels = [r"$\nabla\sigma$ (Non-computed)",
          r"$L_1$ (Non-computed)",
          r"$\mu_i$:" + "Actual computed\nascending direction from $i$"]
          
for i,axi in enumerate(ax):
    axi.xaxis.set_major_locator(ticker.MultipleLocator(1))
    axi.yaxis.set_major_locator(ticker.MultipleLocator(1))

    leg = Legend(axi, [arrs[i]], [labels[i]], loc="upper left", prop={"size": 12}, ncol=1)
    axi.add_artist(leg)

# Show the plot!
plt.show()

In [ ]:
fps = 30
anim = anim_mu_estimation(p, Z, scalar_field, dt=1/fps, fps=fps, tf=3.5, its=15, k=0.1)
HTML(anim.to_html5_video()) # It takes a loooot of time...

# writer = PillowWriter(fps=15, bitrate=1800)
# anim.save(os.path.join(OUTPUT_FOLDER, "ascdir1.gif"),
#         writer = writer)

### Ex 2

In [ ]:
## Parameters #####
pc = np.array([0,0])
lims = np.array([2,2])

np.random.seed(1221)

N = 10
p = XY_distrib(N,pc,lims)
p = p - np.sum(p, axis=0)/N
Z = gen_Z_random(N, rounds=1)

dt = 0.1
its = 50 # iterations per dt

# ----------------------------------------------------------------------
# Generating the scalar field
# ----------------------------------------------------------------------
n = 2
max_int = 20
mu = [10,30]
dev = 10

scalar_field = SigmaNonconvex(k=0.04, dev=dev, mu=mu)

# Draw the scalar field
fig = plt.figure(figsize=(5, 5), dpi=60)
ax = fig.subplots()

scalar_field_plotter = PlotterScalarField(scalar_field)
scalar_field_plotter.draw(fig=fig, ax=ax, xlim=50, ylim=50, contour_levels=8)

# Agents
phi = np.pi/3
for n in range(N):
    icon = unicycle_patch(p[n,:], phi, color="royalblue", size=3)
    ax.add_patch(icon)

# ----------------------------------------------------------------------
# Plotting
# ----------------------------------------------------------------------

fig = plt.figure(figsize=(12, 8), dpi=100)
ax  = fig.subplots(1,3)

plot_mu_estimation(ax[0], p, Z, scalar_field, dt=dt, tf=0.1, its=its, k=2, 
                         legend=False,xlab=True,ylab=True)
plot_mu_estimation(ax[1], p, Z, scalar_field, dt=dt, tf=0.5, its=its, k=2)
plot_mu_estimation(ax[2], p, Z, scalar_field, dt=dt, tf=1, its=its, k=2)

arrs = [plt.scatter([],[],c="k",marker=r"$\uparrow$",s=60),
        plt.scatter([],[],c="blue",marker=r"$\uparrow$",s=60),
        plt.scatter([],[],c="red",marker=r"$\uparrow$",s=60)]

labels = [r"$\nabla\sigma$ (Non-computed)",
          r"$L_1$ (Non-computed)",
          r"$\mu_i$:" + " Actual computed\nascending direction from $i$"]
          
for i,axi in enumerate(ax):
    axi.xaxis.set_major_locator(ticker.MultipleLocator(1))
    axi.yaxis.set_major_locator(ticker.MultipleLocator(1))

    if i == 2:
        fs = 12
    else:
        fs = 14

    leg = Legend(axi, [arrs[i]], [labels[i]], loc="upper left", prop={"size": fs}, ncol=1)
    axi.add_artist(leg)
    axi.set_ylim(np.array(axi.get_ylim())*1.1+0.8)

# Show the plot!
plt.show()

In [ ]:
fps = 30
anim = anim_mu_estimation(p, Z, scalar_field, dt=1/fps, fps=fps, tf=3.5, its=15, k=0.4)
HTML(anim.to_html5_video()) # It takes a loooot of time...

# writer = PillowWriter(fps=15, bitrate=1800)
# anim.save(os.path.join(OUTPUT_FOLDER, "ascdir2.gif"),
#         writer = writer)